In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import scipy as sp

Lets do some data wraggling. We will need to set up martrices to execute a SVD, R=UΣV^T. Normally this is associated with a recommender based on ratings. Here, we will be using the reorder proportions inplace of ratings to predict future reorders. R will be the user reorder matrix. U is user product feature matrix.  Σ is the singular value matrix. V^T is the product feature matrix. 

In [3]:
#loading user order information
instacart_file=pd.read_csv('Data/orders.csv')
df_orders=pd.DataFrame(instacart_file,)
df_orders.head(15)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0
6,550135,1,prior,7,1,9,20.0
7,3108588,1,prior,8,1,14,14.0
8,2295261,1,prior,9,1,16,0.0
9,2550362,1,prior,10,4,8,30.0


Note that the column 'eval_set' breaks down the data into three sets. Details in readme file. What is important to note is that reorders is not provided for the test set (loading into the next dataframe). We there will not be using that set of data. We will add data from the prior set into the train set for are data set. The augmented data set will then be feed into scikit learn model to segregate data into a new train set,cross-validation set, and test set. 

In [4]:
#build data set 
user_order_max=df_orders['order_number'].groupby(df_orders['user_id']).max()
user_order_max.head()

user_id
1    11
2    15
3    13
4     6
5     5
Name: order_number, dtype: int64

In [5]:
user_order_max.size

206209

In [6]:
df_orders.nunique()

order_id                  3421083
user_id                    206209
eval_set                        3
order_number                  100
order_dow                       7
order_hour_of_day              24
days_since_prior_order         31
dtype: int64

In [7]:
#capture the last order information for each user
g = df_orders.groupby('user_id')
data_p1=g.last()

In [8]:
#capture the second to last order information for each user
data_p2=g.nth(-2)

In [9]:
#capture the third to last order information for each user
data_p3=g.nth(-3)

In [10]:
#capture the fourth to last order information for each user
data_p4=g.nth(-4)

In [11]:
#join all information into one data set
data_set=pd.concat([data_p1,data_p2,data_p3,data_p4])

In [12]:
data_set.groupby(['user_id','order_number','eval_set','order_id']).count()

days_since_prior_order  order_dow  \
user_id order_number eval_set order_id                                      
1       8            prior    3108588                        1          1   
        9            prior    2295261                        1          1   
        10           prior    2550362                        1          1   
        11           train    1187899                        1          1   
2       12           prior    3186735                        1          1   
        13           prior    3268552                        1          1   
        14           prior    839880                         1          1   
        15           train    1492625                        1          1   
3       10           prior    676467                         1          1   
        11           prior    521107                         1          1   
        12           prior    1402502                        1          1   
        13           test     2774568                        1          1   
4       3            prior    691089                         1          1   
        4            prior    94891                          1          1   
        5            prior    2557754                        1          1   
        6            test     329954                         1          1   
5       2            prior    1909121                        1          1   
        3            prior    2267326                        1          1   
        4            prior    157374                         1          1   
        5            train    2196797                        1          1   
6       1            prior    2086598                        0          1   
        2            prior    298250                         1          1   
        3            prior    998866                         1          1   
        4            test     1528013                        1          1   
7       18           prior    2984707                        1          1   
        19           prior    369015                         1          1   
        20           prior    2452257                        1          1   
        21           train    525192                         1          1   
8       1            prior    600894                         0          1   
        2            prior    2979257                        1          1   
...                                                        ...        ...   
206202  22           prior    1112560                        1          1   
        23           test     2728930                        1          1   
206203  3            prior    3349091                        1          1   
        4            prior    1333145                        1          1   
        5            prior    133582                         1          1   
        6            train    2371631                        1          1   
206204  2            prior    2511735                        1          1   
        3            prior    79277                          1          1   
        4            prior    1380497                        1          1   
        5            test     350108                         1          1   
206205  1            prior    969311                         0          1   
        2            prior    2658896                        1          1   
        3            prior    414137                         1          1   
        4            train    1716008                        1          1   
206206  65           prior    2750895                        1          1   
        66           prior    2986341                        1          1   
        67           prior    1904200                        1          1   
        68           test     1043943                        1          1   
206207  14           prior    2832273                        1          1   
        15           prior    299117

In [13]:
#remove test set from data
data_set=data_set[data_set.eval_set != 'test']

In [14]:
#set aside new test set
test_set=data_set[data_set.eval_set == 'train']
test_set.nunique()

days_since_prior_order        31
eval_set                       1
order_dow                      7
order_hour_of_day             24
order_id                  131209
order_number                  97
dtype: int64

In [79]:
#assign training set
train_set=data_set[data_set.eval_set == 'prior']
train_set.reset_index(inplace=True)
train_set.nunique()

user_id                   206209
days_since_prior_order        31
eval_set                       1
order_dow                      7
order_hour_of_day             24
order_id                  618627
order_number                  99
dtype: int64

This set is still a bit to big so lets sample a more reasonable portion of users. We will go with 75,ooo users instead. 

In [80]:
train_set=train_set.sample(750)

In [81]:
train_set.reset_index(inplace=True)
train_set.head()

,index,user_id,days_since_prior_order,eval_set,order_dow,order_hour_of_day,order_id,order_number
0,427075,14658,NaN,prior,0,16,1299922,1
1,221438,15230,13.0,prior,2,9,1896063,13
2,493784,81367,7.0,prior,5,12,1792337,7
3,225292,19084,30.0,prior,5,19,1506363,2
4,555227,142810,13.0,prior,6,9,2247593,14


In [82]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750 entries, 0 to 749
Data columns (total 8 columns):
index                     750 non-null int64
user_id                   750 non-null int64
days_since_prior_order    716 non-null float64
eval_set                  750 non-null object
order_dow                 750 non-null int64
order_hour_of_day         750 non-null int64
order_id                  750 non-null int64
order_number              750 non-null int64
dtypes: float64(1), int64(6), object(1)
memory usage: 46.9+ KB


In [19]:
train_set.nunique()

user_id                   750
days_since_prior_order     31
eval_set                    1
order_dow                   7
order_hour_of_day          23
order_id                  750
order_number               75
dtype: int64

# SVD
Now that the data is set up. Lets set up the mechanics for SVD. In short we will need to wraggle our data into dataframes to feed into scripy model. Recall the basic set up is   R=UΣV^T. R we want to be user_id as the index, product_id as the column and reorder rates for users by product as the value.  

In [20]:
#loading product reorder information
instacart_file2=pd.read_csv('Data/order_products__prior.csv')
df_prod_orders=pd.DataFrame(instacart_file2,)
df_prod_orders.head()


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [21]:
#loading information for product names
instacart_products=pd.read_csv('Data/products.csv')
df_prod=pd.DataFrame(instacart_products,)
df_prod.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [22]:
#merge dataframes to get user_id with product_id and reorder in same dataframe
#use inner to get the intersection in order to preserve test set
df_user_order_prod=pd.merge(df_prod_orders,train_set, how= 'inner',left_on="order_id", right_on='order_id')


In [23]:
df_user_order_prod.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,days_since_prior_order,eval_set,order_dow,order_hour_of_day,order_number
0,2682,5959,1,0,86815,30.0,prior,1,11,4
1,2682,49075,2,1,86815,30.0,prior,1,11,4
2,2682,13176,3,0,86815,30.0,prior,1,11,4
3,2682,31292,4,0,86815,30.0,prior,1,11,4
4,2682,22825,5,0,86815,30.0,prior,1,11,4


In [24]:
df_user_order_prod.nunique()

order_id                   750
product_id                3963
add_to_cart_order           55
reordered                    2
user_id                    750
days_since_prior_order      31
eval_set                     1
order_dow                    7
order_hour_of_day           23
order_number                75
dtype: int64

We will want to get reorder rates for user by product.

In [25]:
#count number of product purchases by user
user_products_total=df_user_order_prod.groupby(['user_id','product_id']).size()
user_products_total.head()

user_id  product_id
134      1398          1
         5750          1
         5782          1
         7978          1
         21938         1
dtype: int64

In [26]:
#count number of reorders for user by product
user_item_reorders=df_user_order_prod['reordered'].groupby([df_user_order_prod['user_id'],df_user_order_prod['product_id']]).sum()
user_item_reorders.head()

user_id  product_id
134      1398          0
         5750          1
         5782          1
         7978          0
         21938         1
Name: reordered, dtype: int64

In [27]:
#calculate reorder rate for user by product
user_item_reorder_rate=user_item_reorders/user_products_total
user_item_reorder_rate.rename(columns={0:'reorder_rate'})
user_item_reorder_rate.head()

user_id  product_id
134      1398          0.0
         5750          1.0
         5782          1.0
         7978          0.0
         21938         1.0
dtype: float64

In [28]:
#move series into dataframe and rename columns
df_upr=pd.DataFrame(user_products_total,columns=['prod_order_count'])
df_ur=pd.DataFrame(user_item_reorder_rate,columns=['prod_reorder_rate'])
print(df_upr.head())
print(df_ur.head())
#pd.merge(df_upr.reset_index(), df_ur.reset_index(), on=['user_id'], how='inner').set_index(['user_id','product_id'])

                    prod_order_count
user_id product_id                  
134     1398                       1
        5750                       1
        5782                       1
        7978                       1
        21938                      1
                    prod_reorder_rate
user_id product_id                   
134     1398                      0.0
        5750                      1.0
        5782                      1.0
        7978                      0.0
        21938                     1.0


In [45]:
#join into singe dataframe
df_yes=pd.concat([df_upr, df_ur], axis=1)


In [46]:
df_yes.head()

prod_order_count  prod_reorder_rate
user_id product_id                                     
134     1398                       1                0.0
        5750                       1                1.0
        5782                       1                1.0
        7978                       1                0.0
        21938                      1                1.0

We want products to be the columns, user_id the rows, and the values to be the reorder rate. This will be R, user reorder matrix, for SVD. (after we normalize)

In [31]:
#reset dataframe in order to pivot product_id to columns, user_id to index, and reorder rate to values.
df_reorders=df_yes.reset_index().pivot(index='user_id', columns='product_id', values='prod_reorder_rate')
df_reorders.head()

product_id,10,23,25,34,45,49,63,79,100,108,...,49605,49608,49610,49615,49621,49628,49655,49667,49678,49683
user_id,,,,,,,,,,,,,,,,,,,,,
134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
652,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
787,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1029,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1257,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
#fill NaN with 0 
df_reorders=df_reorders.fillna(0)

In [33]:
df_reorders.info

<bound method DataFrame.info of product_id  10     23     25     34     45     49     63     79     100    \
user_id                                                                     
134           0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
652           0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
787           0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1029          0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1257          0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1954          0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2205          0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2254          0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2521          0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2636          0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3055          0.0    0.0    0.0    0.0    0.

In [34]:
df_reorders.head()

product_id,10,23,25,34,45,49,63,79,100,108,...,49605,49608,49610,49615,49621,49628,49655,49667,49678,49683
user_id,,,,,,,,,,,,,,,,,,,,,
134,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
652,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
787,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1029,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1257,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We will turn that dataframe into a matrix, normalize, optimize the parameters, and make some predictions. 